<a href="https://colab.research.google.com/github/dcastf01/object_detection_TFM/blob/main/Squeeze_para_Istobal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Truco Para que no se cuelgue el chrome, se tiene que incluir en la consola del navegador

function ClickConnect(){ console.log("Working"); document.querySelector("colab-toolbar-button#connect").click() } setInterval(ClickConnect,60000)


#Imports library, dataset and detect device to use

In [ ]:
#@title Imports library
import os, sys, math
import numpy as np
from matplotlib import pyplot as plt
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.4.0


In [ ]:
#@title No necesario si no se quiere guardar información
use_your_drive = False #@param {type:"boolean"}
if use_your_drive:
  from google.colab import drive
  drive.mount('/content/drive')

##download dataset from drive

In [ ]:
#@title Download of dataset
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

# sin_recorte=False #@param {type:"boolean"} 
con_recorte=True #@param {type:"boolean"}  
# con_recorte_y_filtro_frontales=False #@param {type:"boolean"}  
if con_recorte:
  file_id = '1MKjn9qLc2o82Vk1Rua-7TTOmxrOAWi6U'
# if con_recorte_y_filtro_frontales:
#   file_id="13rqJpDl3YKI0oHM0HHwVdifKEw6WP2IX"
# if sin_recorte:
#   file_id="1NiZGvXHs9aQpbciQHwdqsCXJTbbizJap"

destination = '/content/file.zip'

download_file_from_google_drive(file_id, destination)

In [ ]:
#@title Unzip file
%%capture
!unzip file.zip

In [ ]:
#@title Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy for hardware
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  #strategy = tf.distribute.MirroredStrategy(gpus) # this works for 1 to multiple GPUs
  print('Running on ', len(gpus), ' GPU(s) ')
  strategy=tf.distribute.OneDeviceStrategy(gpus[0])
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')

# How many accelerators do we have ?
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on CPU
Number of accelerators:  1


comprobando porque no va la gpu


#Generación del modelo


In [ ]:
#@title Insert setup
import numpy as np
import pathlib

##esto pasarlo al config
BATCH_SIZE = 32 #normalmente en 32 
IMG_HEIGHT = 224
IMG_WIDTH = 224
nb_epochs=100
#dict_data=diccionario_con_datos("squeeze_con_imagenes_en_crudo")
Directory="/content/dataset"
con_recorte=True #@param {type:"boolean"}  
con_batch=True #@param {type:"boolean"}  
con_dropout=False #@param {type:"boolean"} 
if con_recorte:
  Nombre_modelo="Squeeze_con_imagenes_con_recortes"
  if con_batch:
    Nombre_modelo="Squeeze_con_imagenes_con_recortes_filtro_y_Batch"
    if con_dropout:
      value_dropout=0.3 #@param {type:"slider", min:0, max:1, step:0.1}
      Nombre_modelo="Squeeze_con_imagenes_con_recortes_Batch_y_Dropout_solo_sinBypass"+str(value_dropout)
if not con_recorte:
  Nombre_modelo="Squeeze_con_imagenes_sin_recortes"
  if not con_batch or not con_dropout:
    Nombre_modelo="algo raro"
new_metric=True #@param {type:"boolean"}  
if new_metric:
  Nombre_modelo+="read_metrics"

#Nombre_modelo=dict_data["Nombre_modelo"]

data_dir = pathlib.Path(Directory)
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print(CLASS_NAMES)

In [ ]:
#@title Create a dataset
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                               horizontal_flip=True,
                                                                validation_split=0.2)


train_generator  = image_generator.flow_from_directory(directory=str(Directory),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES),
                                                      class_mode="categorical",
                                                    
                                                     subset="training"                                                     
                                                     )
validation_generator  = image_generator.flow_from_directory(directory=str(Directory),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES),
                                                      class_mode="categorical",
                                                     subset="validation"                                                     
                                                     )
print("number of images en train",train_generator.n)
print("number of images en validation",validation_generator.n)

# Model


In [ ]:
#@title Generate model

IMAGE_SIZE=[227,227] #en el experimento usaron 227
NUM_CLASS=train_generator.num_classes
with strategy.scope(): # this line is all that is needed to run on TPU (or multi-GPU, ...)

  bnmomemtum=0.9
  def fire(x, squeeze, expand):
   
    if con_batch:
      x=tf.keras.layers.BatchNormalization()(x)
    if con_dropout:
      x=tf.keras.layers.Dropout(value_dropout)(x)
    y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    
    y1 = tf.keras.layers.Conv2D(filters=expand, kernel_size=1, activation='relu', padding='same')(y)
 
    y3 = tf.keras.layers.Conv2D(filters=expand, kernel_size=3, activation='relu', padding='same')(y)
   
   
    return tf.keras.layers.concatenate([y1, y3])

  def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)
    
  def fire_with_bypass(x, squeeze, expand):
    
    if con_batch:
      x=tf.keras.layers.BatchNormalization()(x)
    #if con_dropout:
     # x=tf.keras.layers.Dropout(value_dropout)(x)
    y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)

    y1 = tf.keras.layers.Conv2D(filters=expand, kernel_size=1, activation='relu', padding='same')(y)
 
    y3 = tf.keras.layers.Conv2D(filters=expand, kernel_size=3, activation='relu', padding='same')(y)
   
    y5=tf.keras.layers.concatenate([y1, y3])
    return tf.keras.layers.add([x, y5])

  def fire_module_with_bypass(squeeze, expand):
    return lambda x: fire_with_bypass(x, squeeze, expand)
 
  x = tf.keras.layers.Input(shape=[*IMAGE_SIZE, 3]) # input is 192x192 pixels RGB

  y = tf.keras.layers.Conv2D(kernel_size=3, filters=95,strides=2, use_bias=True, activation='relu')(x)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module(16, 64)(y)
  y = fire_module_with_bypass(16, 64)(y)
  y = fire_module(32, 128)(y)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module_with_bypass(32, 128)(y)
  y = fire_module(48, 192)(y)
  y = fire_module_with_bypass(48, 192)(y)
  y = fire_module(64, 256)(y)
  y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
  y = fire_module_with_bypass(64, 256)(y)
  
  if con_batch:
    y=tf.keras.layers.BatchNormalization()(y)
  if con_dropout:
      y=tf.keras.layers.Dropout(value_dropout)(y)
  y = tf.keras.layers.Conv2D(kernel_size=3, filters=NUM_CLASS,strides=2, use_bias=True, activation='relu')(y) #en el modelo el numero de filtros es igual al número de clases
  y = tf.keras.layers.GlobalAveragePooling2D()(y)
  y = tf.keras.layers.Dense(NUM_CLASS, activation='softmax')(y) 

  #revisar para disminuir el número de clases


  model = tf.keras.Model(x, y)
 # initiate RMSprop optimizer
  opt = tf.keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)

  METRICS=[ 
      
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      tf.keras.metrics.Accuracy(name="accuracy_binary"),
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc',num_thresholds=500),#cambiar el auc
      tf.keras.metrics.TopKCategoricalAccuracy( k=3, name='top_3_categorical_accuracy'),
      tf.keras.metrics.TopKCategoricalAccuracy( k=5, name='top_5_categorical_accuracy'),
      tf.keras.metrics.TopKCategoricalAccuracy( k=10, name='top_10_categorical_accuracy')
        ]
    # Let's train the model using RMSprop
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=METRICS)
 #model.summary()

In [ ]:
#@title if you execute the cell you have a plot of model
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
#@title create different callbacks
#la variable model_dir tiene que terminar con "/"

model_dir = "/content/tensorboard/" #@param {type:"string"}


def compilacion(model,Nombre_modelo="faltonombre"):
    from tensorflow import keras
    from tensorflow.keras.optimizers import SGD
    
   
    from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
    from tensorflow.keras.callbacks import ReduceLROnPlateau

   
    import datetime
    verbose=1
    patience = 50

    #log_dir="logs/"+Nombre_modelo+"/tensorboard/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir=model_dir+"logs/"+Nombre_modelo+"/tensorboard"
    tensor_board = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
    log_file_path = model_dir+'logs/'+Nombre_modelo+'/history.log'
    csv_logger = CSVLogger(log_file_path, append=True)
    early_stop = EarlyStopping('val_accuracy', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_accuracy', factor=0.1, patience=int(patience / 4), verbose=1)
    trained_models_path = model_dir+'models/'
    model_names = trained_models_path + Nombre_modelo+"/"+'primerTest{epoch:02d}val_accuracy{val_accuracy:.2f}time'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)
    callbacks = [tensor_board, model_checkpoint, csv_logger, early_stop, reduce_lr]
    
    return callbacks
callbacks=compilacion(model,Nombre_modelo)


In [ ]:
#@title load model pre-trained
#usar solo si se dispone de alguna versióon del modelo entrenada
use_model_pre_trained = False #@param {type:"boolean"}

if use_model_pre_trained:
  model_pretraine_hdf5=""#@param {type:"string"}
  model.load_weights("/content/drive/My Drive/AI/Computer vision/car recognition/framework_label/models/Squeeze_imagenes_en_crudo/primerTest52val_accuracy0.46time20200326-081652.hdf5")

#Train


In [ ]:
#@title train model
history=model.fit(
    train_generator,
    steps_per_epoch =( train_generator.samples // BATCH_SIZE),
    validation_data = validation_generator, 
    validation_steps = (validation_generator.samples // BATCH_SIZE),
    epochs = 100,
    callbacks=callbacks
    )

#tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
model_tensorboard=model_dir+"/logs"

In [ ]:
%tensorboard --logdir {model_tensorboard}
